## Homework

> Note: sometimes your answer doesn't match one of 
> the options exactly. That's fine. 
> Select the option that's closest to your solution.

### Dataset

In this homework, we will use the California Housing Prices from [Kaggle](https://www.kaggle.com/datasets/camnugent/california-housing-prices).

Here's a wget-able [link](https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv):

```bash
wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv
```

The goal of this homework is to create a regression model for predicting housing prices (column `'median_house_value'`).

### EDA

* Load the data.
* Look at the `median_house_value` variable. Does it have a long tail?

In [250]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [251]:
df = pd.read_csv("dataset/housing.csv")
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB
None


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


### Preparing the dataset 

For this homework, we only want to use a subset of data. 

First, keep only the records where `ocean_proximity` is either `'<1H OCEAN'` or `'INLAND'`

Next, use only the following columns:

* `'latitude'`,
* `'longitude'`,
* `'housing_median_age'`,
* `'total_rooms'`,
* `'total_bedrooms'`,
* `'population'`,
* `'households'`,
* `'median_income'`,
* `'median_house_value'`

In [252]:
proxi_mask = (df["ocean_proximity"] == '<1H OCEAN') | (df["ocean_proximity"] == 'INLAND')
subset_df = df[proxi_mask].drop("ocean_proximity", axis=1).reset_index().drop("index", axis=1)
subset_df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-121.97,37.64,32.0,1283.0,194.0,485.0,171.0,6.0574,431000.0
1,-121.99,37.61,9.0,3666.0,711.0,2341.0,703.0,4.6458,217000.0
2,-121.97,37.57,21.0,4342.0,783.0,2172.0,789.0,4.6146,247600.0
3,-121.96,37.58,15.0,3575.0,597.0,1777.0,559.0,5.7192,283500.0
4,-121.98,37.58,20.0,4126.0,1031.0,2079.0,975.0,3.6832,216900.0
...,...,...,...,...,...,...,...,...,...
15682,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0
15683,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0
15684,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0
15685,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0


### Question 1

There's one feature with missing values. What is it?

* `total_rooms`
* `total_bedrooms`
* `population`
* `households`

In [253]:
subset_df.isna().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        157
population              0
households              0
median_income           0
median_house_value      0
dtype: int64

In [254]:
# feature with missing values : total_bedrooms

### Question 2

What's the median (50% percentile) for variable `'population'`?

- 995
- 1095
- 1195
- 1295

In [255]:
subset_df["population"].median()

1195.0

In [256]:
# the median (50% percentile) for variable 'population' : 1195

### Prepare and split the dataset

* Shuffle the dataset (the filtered one you created above), use seed `42`.
* Split your data in train/val/test sets, with 60%/20%/20% distribution.
* Apply the log transformation to the `median_house_value` variable using the `np.log1p()` function.

In [257]:
n = len(subset_df)
n_val = int(n * 0.2)
n_test = int(n * 0.2)
n_train = n - n_val - n_test

In [258]:
# Shuffle the dataset (the filtered one you created above), use seed 42
idx = np.arange(n)
np.random.seed(42)
np.random.shuffle(idx)
df_shuffled = subset_df.iloc[idx]

In [259]:
# Split your data in train/val/test sets, with 60%/20%/20% distribution.
df_train = df_shuffled.iloc[:n_train].copy()
df_val = df_shuffled.iloc[n_train:n_train + n_val].copy()
df_test = df_shuffled.iloc[n_train + n_val:].copy()

In [260]:
df_train.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
15183,-119.14,36.23,22.0,2935.0,523.0,1927.0,530.0,2.5875,70400.0
4469,-117.79,34.12,16.0,2426.0,426.0,1319.0,446.0,4.8125,224500.0
9316,-117.97,33.68,26.0,3653.0,568.0,1930.0,585.0,5.7301,260900.0
4983,-118.03,34.10,32.0,2668.0,609.0,1512.0,541.0,2.9422,233100.0
13154,-121.87,37.34,39.0,2479.0,541.0,1990.0,506.0,2.4306,289100.0


In [261]:
len(df_train), len(df_val), len(df_test)

(9413, 3137, 3137)

In [262]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [263]:
y_train_ori = df_train['median_house_value'].values
y_val_ori = df_val['median_house_value'].values
y_test_ori = df_test['median_house_value'].values

In [264]:
# Apply the log transformation to the median_house_value variable using the np.log1p() function.
y_train = np.log1p(df_train['median_house_value'].values)
y_val = np.log1p(df_val['median_house_value'].values)
y_test = np.log1p(df_test['median_house_value'].values)

In [265]:
# delete before transformation before using machine learning
del df_train["median_house_value"]
del df_val["median_house_value"]
del df_test["median_house_value"]

In [266]:
assert len(df_train) == len(y_train)
assert len(df_val) == len(y_val)
assert len(df_test) == len(y_test)

In [267]:
# Linear Regression
def train_linear_regression(X, y):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    XTX_inv = np.linalg.inv(XTX)
    w = XTX_inv.dot(X.T).dot(y)

    return w[0], w[1:]

### Question 3

* We need to deal with missing values for the column from Q1.
* We have two options: fill it with 0 or with the mean of this variable.
* Try both options. For each, train a linear regression model without regularization using the code from the lessons.
* For computing the mean, use the training only!
* Use the validation dataset to evaluate the models and compare the RMSE of each option.
* Round the RMSE scores to 2 decimal digits using `round(score, 2)`
* Which option gives better RMSE?

In [268]:
total_bedrooms_before = df_train["total_bedrooms"]
total_bedrooms_before

0       523.0
1       426.0
2       568.0
3       609.0
4       541.0
        ...  
9408    392.0
9409    385.0
9410    147.0
9411    890.0
9412    441.0
Name: total_bedrooms, Length: 9413, dtype: float64

In [269]:
# fill missing values with mean
df_train["total_bedrooms"] = df_train["total_bedrooms"].fillna(df_train["total_bedrooms"].mean())
df_val["total_bedrooms"] = df_val["total_bedrooms"].fillna(df_val["total_bedrooms"].mean())

In [270]:
# Baseline
def prepare_X(df):
    X = df.values
    return X

In [271]:
X_val = prepare_X(df_val)
w_0, w = train_linear_regression(X_train, y_train)

In [272]:
y_pred = w_0 + X_val.dot(w)

In [273]:
def rmse(y, y_pred):
    error = y_pred - y
    mse = (error ** 2).mean()
    return np.sqrt(mse)

In [274]:
round(rmse(y_val, y_pred), 2)

0.34

In [275]:
# fill missing values with 0
df_train['total_bedrooms'] = total_bedrooms_before.fillna(0)
df_val['total_bedrooms'] = total_bedrooms_before.fillna(0)

In [276]:
X_val = prepare_X(df_val)
w_0, w = train_linear_regression(X_train, y_train)

In [277]:
y_pred = w_0 + X_val.dot(w)

In [278]:
round(rmse(y_val, y_pred), 2)

0.39

In [279]:
# better RMSE is with fillna 0

### Question 4

* Now let's train a regularized linear regression.
* For this question, fill the NAs with 0. 
* Try different values of `r` from this list: `[0, 0.000001, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10]`.
* Use RMSE to evaluate the model on the validation dataset.
* Round the RMSE scores to 2 decimal digits.
* Which `r` gives the best RMSE?

If there are multiple options, select the smallest `r`.

Options:

- 0
- 0.000001
- 0.001
- 0.0001


In [280]:
def train_linear_regression_reg(X, y, r):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    
    XTX = X.T.dot(X)
    reg = r * np.eye(XTX.shape[0])
    XTX = XTX + reg

    XTX_inv = np.linalg.inv(XTX)
    w = XTX_inv.dot(X.T).dot(y)
    
    return w[0], w[1:]

In [281]:
X_train = prepare_X(df_train)

In [282]:
r_values = [0, 0.000001, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10]

for r in r_values:
    w_0, w = train_linear_regression_reg(X_train, y_train, r=r)
    X_val = prepare_X(df_val)
    y_pred = w_0 + X_val.dot(w)
    print(f"r: {r}, val: {round(rmse(y_val, y_pred), 2)}")

r: 0, val: 0.39
r: 1e-06, val: 0.39
r: 0.0001, val: 0.39
r: 0.001, val: 0.39
r: 0.01, val: 0.38
r: 0.1, val: 0.38
r: 1, val: 0.37
r: 5, val: 0.37
r: 10, val: 0.37


In [283]:
# r that gives the best RMSE is 0.000001

### Question 5 

* We used seed 42 for splitting the data. Let's find out how selecting the seed influences our score.
* Try different seed values: `[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]`.
* For each seed, do the train/validation/test split with 60%/20%/20% distribution.
* Fill the missing values with 0 and train a model without regularization.
* For each seed, evaluate the model on the validation dataset and collect the RMSE scores. 
* What's the standard deviation of all the scores? To compute the standard deviation, use `np.std`.
* Round the result to 3 decimal digits (`round(std, 3)`)

What's the value of std?

- 0.5
- 0.05
- 0.005
- 0.0005

> Note: Standard deviation shows how different the values are.
> If it's low, then all values are approximately the same.
> If it's high, the values are different. 
> If standard deviation of scores is low, then our model is *stable*.

In [288]:
seeds = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
rmse_scores = []
idx = np.arange(n)
for seed in seeds:
    # shuffled data using different seeds
    np.random.seed(seed)
    np.random.shuffle(idx)
    df_shuffled = subset_df.iloc[idx]

    # split the data
    df_train = df_shuffled.iloc[:n_train].copy()
    df_val = df_shuffled.iloc[n_train:n_train + n_val].copy()
    df_test = df_shuffled.iloc[n_train + n_val:].copy()

    # determine the target
    y_train = np.log1p(df_train['median_house_value'].values)
    y_val = np.log1p(df_val['median_house_value'].values)
    y_test = np.log1p(df_test['median_house_value'].values)

    # delete target from feature dataset
    del df_train["median_house_value"]
    del df_val["median_house_value"]
    del df_test["median_house_value"]

    # fill missing values with 0
    df_train['total_bedrooms'] = df_train["total_bedrooms"].fillna(0)
    df_val['total_bedrooms'] = df_val["total_bedrooms"].fillna(0)

    # train model using train dataset and target
    w_0, w = train_linear_regression(X_train, y_train)

    # predict the target using validation dataset
    X_val = prepare_X(df_val)
    y_pred = w_0 + X_val.dot(w)

    # append each rmse to rrmse_scores list
    rmse_scores.append(round(rmse(y_val, y_pred), 2))

In [290]:
rmse_scores

[0.56, 0.57, 0.56, 0.56, 0.55, 0.57, 0.56, 0.56, 0.56, 0.57]

In [291]:
round(np.std(rmse_scores), 3)

0.006

In [292]:
# the closest value of std = 0.005

### Question 6

* Split the dataset like previously, use seed 9.
* Combine train and validation datasets.
* Fill the missing values with 0 and train a model with `r=0.001`. 
* What's the RMSE on the test dataset?

Options:

- 0.13
- 0.23
- 0.33
- 0.43

In [305]:
# shuffled data using seed = 9
np.random.seed(9)
np.random.shuffle(idx)
df_shuffled = subset_df.iloc[idx]

# split the data
n = len(subset_df)
n_test = int(n * 0.2)
n_train = n - n_test

df_train = df_shuffled.iloc[:n_train].copy()
df_test = df_shuffled.iloc[n_train:].copy()

# determine the target
y_train = np.log1p(df_train['median_house_value'].values)
y_test = np.log1p(df_test['median_house_value'].values)

# delete target from feature dataset
del df_train["median_house_value"]
del df_test["median_house_value"]

# fill missing values with 0
df_train['total_bedrooms'] = df_train["total_bedrooms"].fillna(0)
df_test['total_bedrooms'] = df_test["total_bedrooms"].fillna(0)

# train model using train dataset and target using r=0.001
X_train = prepare_X(df_train)
w_0, w = train_linear_regression_reg(X_train, y_train, r=0.001)

# # predict the target using validation dataset
X_test = prepare_X(df_test)
y_pred = w_0 + X_test.dot(w)

rmse = rmse(y_test, y_pred)

In [307]:
round(rmse, 2)

0.34